Ref: https://github.com/SchindlerLiang/VAE-for-Anomaly-Detection

In [1]:
import os
import random

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID";
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ["TF_FORCE_GPU_ALLOW_GROWTH"] = "true"

In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf

from evaluator import evaluate
from data_loader import load_kdd_cup_urc, load_yahoo_A1, load_yahoo_A2, load_yahoo_A3, load_yahoo_A4, load_power_demand # Univariate Datasets
from data_loader import load_nasa, load_ecg, load_gesture, load_smd # Multivariate Datasets

from tensorflow import keras
from tensorflow.keras import layers
from tqdm.notebook import tqdm

# THESE LINES ARE FOR REPRODUCIBILITY
random.seed(0)
np.random.seed(0)
tf.random.set_seed(0)

In [3]:
class Sampling(layers.Layer):
    """Uses (z_mean, z_log_var) to sample z, the vector encoding a digit."""

    def call(self, inputs):
        z_mean, z_log_var = inputs
        batch = tf.shape(z_mean)[0]
        dim = tf.shape(z_mean)[1]
        epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon

In [4]:
class VAE(keras.Model):
    def __init__(self, encoder, decoder, **kwargs):
        super(VAE, self).__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder
        self.total_loss_tracker = keras.metrics.Mean(name="total_loss")
        self.reconstruction_loss_tracker = keras.metrics.Mean(
            name="reconstruction_loss"
        )
        self.kl_loss_tracker = keras.metrics.Mean(name="kl_loss")

    @property
    def metrics(self):
        return [
            self.total_loss_tracker,
            self.reconstruction_loss_tracker,
            self.kl_loss_tracker,
        ]

    def train_step(self, data):
        with tf.GradientTape() as tape:
            z_mean, z_log_var, z = self.encoder(data)
            reconstruction = self.decoder(z)
            reconstruction_loss = tf.reduce_mean(
                tf.reduce_sum(
                    keras.losses.binary_crossentropy(data, reconstruction), axis=1
                )
            )
            kl_loss = -0.5 * (1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var))
            kl_loss = tf.reduce_mean(tf.reduce_sum(kl_loss, axis=1))
            total_loss = reconstruction_loss + kl_loss
        grads = tape.gradient(total_loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
        self.total_loss_tracker.update_state(total_loss)
        self.reconstruction_loss_tracker.update_state(reconstruction_loss)
        self.kl_loss_tracker.update_state(kl_loss)
        return {
            "loss": self.total_loss_tracker.result(),
            "reconstruction_loss": self.reconstruction_loss_tracker.result(),
            "kl_loss": self.kl_loss_tracker.result(),
        }

In [5]:
def GRU_VAE(X_train):
    latent_dim = 16

    encoder_inputs = keras.Input(shape=(X_train.shape[1], X_train.shape[2]))
    x = layers.GRU(64, return_sequences=True)(encoder_inputs)
    x = layers.GRU(32)(x)
    z_mean = layers.Dense(latent_dim, name="z_mean")(x)
    z_log_var = layers.Dense(latent_dim, name="z_log_var")(x)
    z = Sampling()([z_mean, z_log_var])
    encoder = keras.Model(encoder_inputs, [z_mean, z_log_var, z], name="encoder")

    latent_inputs = keras.Input(shape=(latent_dim,))
    x = layers.RepeatVector(X_train.shape[1])(latent_inputs)
    x = layers.GRU(32, return_sequences=True)(x)
    x = layers.GRU(64)(x)
    x = layers.Dense(X_train.shape[1] *  X_train.shape[2])(x)
    decoder_outputs = layers.Reshape([X_train.shape[1], X_train.shape[2]])(x)
    decoder = keras.Model(latent_inputs, decoder_outputs, name="decoder")

    model = VAE(encoder, decoder)
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001))
    history = model.fit(X_train, epochs=50, batch_size=128, verbose=0)
    return model

### Yahoo S5

In [6]:
total_scores = {'dataset': [], 'f1': [], 'pr_auc': [], 'roc_auc': []}

In [7]:
for loader in [load_yahoo_A1, load_yahoo_A2, load_yahoo_A3, load_yahoo_A4]:
    datasets = loader(8, 4)
    x_trains, x_tests, y_tests = datasets['x_train'], datasets['x_test'], datasets['y_test']
    
    for i in tqdm(range(len(x_trains))):
        tf.keras.backend.clear_session()

        X_train = x_trains[i]
        X_test = x_tests[i]
        
        model = GRU_VAE(X_train)
        
        X_test_rec = model.decoder.predict(model.encoder.predict(X_test)[-1])
        scores = evaluate(X_test, X_test_rec, y_tests[i], is_reconstructed=True)
    
        total_scores['dataset'].append(loader.__name__.replace('load_', ''))
        total_scores['f1'].append(np.max(scores['f1']))
        total_scores['pr_auc'].append(scores['pr_auc'])
        total_scores['roc_auc'].append(scores['roc_auc'])
        print(loader.__name__.replace('load_', ''), np.max(scores['f1']), scores['pr_auc'], scores['roc_auc'])   

  0%|          | 0/67 [00:00<?, ?it/s]

  0%|          | 0/56 [00:00<?, ?it/s]

yahoo_A1 0.9999999000000052 0.4999999375000063 0.9999999490291289
yahoo_A1 0.9999999166666703 0.6666666055555603 0.9999999656957942
yahoo_A1 0.9090908429752097 0.7423194671186263 0.9774999827308337
yahoo_A1 0.9999998500000123 0.0 0.9999998990476291
yahoo_A1 0.666666577777783 0.03170289683151527 0.8942307236593959
yahoo_A1 0.09090907355372142 0.022157008474224395 0.5432692030833964
yahoo_A1 0.9999999166666703 0.6666666055555603 0.999999965714287
yahoo_A1 0.6440677444412564 0.6287295542043828 0.740582371953435
yahoo_A1 0.602409590942085 0.577141538891256 0.7695555519642964
yahoo_A1 0.999999930000003 0.7999999503333362 0.9999999790099015
yahoo_A1 0.4999999500000025 0.19663532335309997 0.5337541973709274
yahoo_A1 0.999999930000003 0.7999999503333362 0.9999999790000005
yahoo_A1 0.9999999437500025 0.9374999763313819 0.9999999926264045
yahoo_A1 0.05555555005144069 0.004854369563728194 0.004901960289311856
yahoo_A1 0.13793100499406027 0.05390465393664852 0.6135922190256389
yahoo_A1 0.999999940

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

...

  0%|          | 0/92 [00:00<?, ?it/s]

yahoo_A3 0.03174602857142873 0.004065040847075319 -0.0
yahoo_A3 0.12121210964187425 0.04046672576162125 0.2980295523050943
yahoo_A3 0.28571425306122533 0.03718395698731636 0.5639830389697885
yahoo_A3 0.03174602857142873 0.004065040847075319 -0.0
yahoo_A3 0.5333332817777814 0.36370934953337986 0.8196811358754239
yahoo_A3 0.0624999938476567 0.006777973545986244 0.08958332877951414
yahoo_A3 0.17647057188581453 0.03576215507256644 0.4571135372839822
yahoo_A3 0.09523807770219492 0.03671193935490493 0.43926552765975635
yahoo_A3 0.2352940692041611 0.04965386806381714 0.4231638307991082
yahoo_A3 0.14925371730897866 0.03525299135920128 0.3167815320568335
yahoo_A3 0.3999999502222271 0.1516666607131358 0.5942843135550926
yahoo_A3 0.19999995422223202 0.06727896856528959 0.4628950785000059
yahoo_A3 0.07407406666666702 0.021461788723660415 0.6516393111461991
yahoo_A3 0.17647057188581453 0.049206096676681715 0.3193540918841009
yahoo_A3 0.04724408980097991 0.004132231936856748 0.008264461976641038
yah

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/82 [00:00<?, ?it/s]

yahoo_A4 0.09230768336094745 0.03464462179309658 0.44378530059754573
yahoo_A4 0.034782605217391474 0.005952381015748814 0.31967210342650076
yahoo_A4 0.22222216790124563 0.03873586440980373 0.45972220450115814
yahoo_A4 0.0624999938476567 0.011332704662679998 0.16597221471122725
yahoo_A4 0.7142856551020442 0.546228235266054 0.8035714174843863
yahoo_A4 0.5714285142857186 0.3059334374600662 0.8098316788230889
yahoo_A4 0.07692305029586642 0.01534694068195465 0.41874998110937595
yahoo_A4 0.07751937227330145 0.015211913991676006 0.20518206611492656
yahoo_A4 0.10714283762755424 0.03375383571957729 0.4574152427716117
yahoo_A4 0.11764702145329789 0.029117999477114735 0.4763888737233801
yahoo_A4 0.14634144544913943 0.050124423955070815 0.5067220779233278
yahoo_A4 0.0624999938476567 0.007390288949763428 0.17916665853125044
yahoo_A4 0.18181813223141666 0.05727452471617864 0.7374999684131959
yahoo_A4 0.09230768336094745 0.025619552284913867 0.4689265403417765
yahoo_A4 0.03174602857142873 0.004098360

In [8]:
yahoo_results = pd.DataFrame(total_scores)
yahoo_results.groupby('dataset').mean()

,f1,pr_auc,roc_auc
dataset,,,
yahoo_A1,0.570556,0.364893,0.745028
yahoo_A2,0.625884,0.342117,0.841285
yahoo_A3,0.171302,0.067527,0.436916
yahoo_A4,0.161248,0.054900,0.435486


### NASA

In [9]:
total_scores = {'dataset': [], 'f1': [], 'pr_auc': [], 'roc_auc': []}

In [10]:
for loader in [load_nasa]:
    datasets = loader(8, 4)
    x_trains, x_tests, y_tests = datasets['x_train'], datasets['x_test'], datasets['y_test']
    
    for i in tqdm(range(len(x_trains))):
        tf.keras.backend.clear_session()

        X_train = x_trains[i]
        X_test = x_tests[i]
        
        model = GRU_VAE(X_train)
        
        X_test_rec = model.decoder.predict(model.encoder.predict(X_test)[-1])
        scores = evaluate(X_test, X_test_rec, y_tests[i], is_reconstructed=True)
        
        total_scores['dataset'].append(f'D{i+1}')
        total_scores['f1'].append(np.max(scores['f1']))
        total_scores['pr_auc'].append(scores['pr_auc'])
        total_scores['roc_auc'].append(scores['roc_auc'])
        print(f'D{i+1}', np.max(scores['f1']), scores['pr_auc'], scores['roc_auc'])   

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

D1 0.22955523929021127 0.11037064583849406 0.42164889157010427
D2 0.19498578202833744 0.29016967077669653 0.529774639992712


In [11]:
nasa_results = pd.DataFrame(total_scores)
nasa_results.groupby('dataset').mean()

,f1,pr_auc,roc_auc
dataset,,,
D1,0.229555,0.110371,0.421649
D2,0.194986,0.290170,0.529775


### SMD

In [12]:
total_scores = {'dataset': [], 'f1': [], 'pr_auc': [], 'roc_auc': []}

In [13]:
for loader in [load_smd]:
    datasets = loader(8, 4)
    x_trains, x_tests, y_tests = datasets['x_train'], datasets['x_test'], datasets['y_test']
    
    for i in tqdm(range(len(x_trains))):
        tf.keras.backend.clear_session()

        X_train = x_trains[i]
        X_test = x_tests[i]
        
        model = GRU_VAE(X_train)
        
        X_test_rec = model.decoder.predict(model.encoder.predict(X_test)[-1])
        scores = evaluate(X_test, X_test_rec, y_tests[i], is_reconstructed=True)

        total_scores['dataset'].append(loader.__name__.replace('load_', ''))
        total_scores['f1'].append(np.max(scores['f1']))
        total_scores['pr_auc'].append(scores['pr_auc'])
        total_scores['roc_auc'].append(scores['roc_auc'])
        print(loader.__name__.replace('load_', ''), np.max(scores['f1']), scores['pr_auc'], scores['roc_auc'])   

  0%|          | 0/28 [00:00<?, ?it/s]

  0%|          | 0/28 [00:00<?, ?it/s]

smd 0.17627464443920743 0.5532991977145021 0.5087209301507045
smd 0.5753424154742959 0.4713858630970242 0.9152976207707749
smd 0.15455743881291528 0.10972946736224312 0.6664374861059538
smd 0.20689651046539334 0.139833472498815 0.6399247950521386
smd 0.6551723644470901 0.6469868170827895 0.9002683886463981
smd 0.8390128769844313 0.8389050323100404 0.8948808551036604
smd 0.19097447482898391 0.15598504621626064 0.519048120718654
smd 0.26562497675781443 0.21047019988337137 0.7191300361036563
smd 0.43797851044138947 0.37109511874052176 0.7464612711699085
smd 0.21837868410447125 0.20723977928088938 0.5095047152384006
smd 0.515723220030858 0.5604417666307124 0.7390910364923281
smd 0.31598510338787694 0.3305600515774486 0.775751921785226
smd 0.2626262206560622 0.2458829679143119 0.7891266882744127
smd 0.4585986888393067 0.37930113788115116 0.6570459735071392
smd 0.6814158814550898 0.5991029507879582 0.8937030684286914
smd 0.9499999477500025 0.8942184365553694 0.9760487986827799
smd 0.52229295

In [14]:
smd_results = pd.DataFrame(total_scores)
smd_results.groupby('dataset').mean()

,f1,pr_auc,roc_auc
dataset,,,
smd,0.442602,0.437724,0.751747


### ECG

In [15]:
total_scores = {'dataset': [], 'f1': [], 'pr_auc': [], 'roc_auc': []}

In [16]:
for loader in [load_ecg]:
    datasets = loader(4, 2)
    x_trains, x_tests, y_tests = datasets['x_train'], datasets['x_test'], datasets['y_test']
    
    for i in tqdm(range(len(x_trains))):
        X_train = x_trains[i]
        X_test = x_tests[i]
        
        model = GRU_VAE(X_train)
        
        X_test_rec = model.decoder.predict(model.encoder.predict(X_test)[-1])
        scores = evaluate(X_test, X_test_rec, y_tests[i], is_reconstructed=True)

        total_scores['dataset'].append(f'D{i+1}')
        total_scores['f1'].append(np.max(scores['f1']))
        total_scores['pr_auc'].append(scores['pr_auc'])
        total_scores['roc_auc'].append(scores['roc_auc'])
        print(f'D{i+1}', np.max(scores['f1']), scores['pr_auc'], scores['roc_auc'])  

  0%|          | 0/9 [00:00<?, ?it/s]

  0%|          | 0/9 [00:00<?, ?it/s]

D1 0.40167359078448106 0.4246829572525 0.6314928775117596
D2 0.33082701954887905 0.29317211650272645 0.5915459048439115
D3 0.15720519569040742 0.06256919775317767 0.5576123414902308
D4 0.2544528981320725 0.16023626864225748 0.6131614149477643
D5 0.3636363195909144 0.248091004678139 0.7915607302799067
D6 0.16102468660033353 0.09061489733851968 0.48902827627804557
D7 0.054740951658759245 0.019293964926527508 0.5285622275347569
D8 0.17595700206168416 0.084210497535342 0.4706210185437036
D9 0.3584905364259547 0.15594278926258578 0.3373023271329093


In [17]:
ecg_results = pd.DataFrame(total_scores)
ecg_results.groupby('dataset').mean()

,f1,pr_auc,roc_auc
dataset,,,
D1,0.401674,0.424683,0.631493
D2,0.330827,0.293172,0.591546
D3,0.157205,0.062569,0.557612
D4,0.254453,0.160236,0.613161
D5,0.363636,0.248091,0.791561
D6,0.161025,0.090615,0.489028
D7,0.054741,0.019294,0.528562
D8,0.175957,0.084210,0.470621
D9,0.358491,0.155943,0.337302


### Power Demand

In [18]:
total_scores = {'dataset': [], 'f1': [], 'pr_auc': [], 'roc_auc': []}

In [19]:
for loader in [load_power_demand]:
    datasets = loader(16, 8)
    x_trains, x_tests, y_tests = datasets['x_train'], datasets['x_test'], datasets['y_test']
    
    for i in tqdm(range(len(x_trains))):
        X_train = x_trains[i]
        X_test = x_tests[i]
        
        model = GRU_VAE(X_train)
        
        X_test_rec = model.decoder.predict(model.encoder.predict(X_test)[-1])
        scores = evaluate(X_test, X_test_rec, y_tests[i], is_reconstructed=True)

        total_scores['dataset'].append(loader.__name__.replace('load_', ''))
        total_scores['f1'].append(np.max(scores['f1']))
        total_scores['pr_auc'].append(scores['pr_auc'])
        total_scores['roc_auc'].append(scores['roc_auc'])
        print(loader.__name__.replace('load_', ''), np.max(scores['f1']), scores['pr_auc'], scores['roc_auc']) 

  0%|          | 0/1 [00:00<?, ?it/s]

power_demand 0.26431715744532397 0.11515928338440881 0.41098443550840974


In [20]:
power_results = pd.DataFrame(total_scores)
power_results.groupby('dataset').mean()

,f1,pr_auc,roc_auc
dataset,,,
power_demand,0.264317,0.115159,0.410984


### 2D Gesture

In [21]:
total_scores = {'dataset': [], 'f1': [], 'pr_auc': [], 'roc_auc': []}

In [22]:
for loader in [load_gesture]:
    datasets = loader(4, 2)
    x_trains, x_tests, y_tests = datasets['x_train'], datasets['x_test'], datasets['y_test']
    
    for i in tqdm(range(len(x_trains))):
        X_train = x_trains[i]
        X_test = x_tests[i]

        model = GRU_VAE(X_train)
        
        X_test_rec = model.decoder.predict(model.encoder.predict(X_test)[-1])
        scores = evaluate(X_test, X_test_rec, y_tests[i], is_reconstructed=True)

        total_scores['dataset'].append(loader.__name__.replace('load_', ''))
        total_scores['f1'].append(np.max(scores['f1']))
        total_scores['pr_auc'].append(scores['pr_auc'])
        total_scores['roc_auc'].append(scores['roc_auc'])
        print(loader.__name__.replace('load_', ''), np.max(scores['f1']), scores['pr_auc'], scores['roc_auc'])  

  0%|          | 0/1 [00:00<?, ?it/s]

gesture 0.41666663302103574 0.2445769392388267 0.4980060051513198


In [23]:
gesture_results = pd.DataFrame(total_scores)
gesture_results.groupby('dataset').mean()

,f1,pr_auc,roc_auc
dataset,,,
gesture,0.416667,0.244577,0.498006
